In [0]:
import numpy as np
import pandas as pd

class BlackjackGame:

    ###
    # The constructor for the game. Defining how many decks are included for play and penetration (where the game ends).
    # The default is 6 decks and a game ends when the dealer has dealt 1.5 times the number of cards in the deck.
    # Initialized the randomized stack of cards and it is ready for play.
    def __init__(self, num_of_decks=6, penetration=1.5, seed=42):
        self.num_of_decks = num_of_decks
        self.penetration = penetration
        self.deck_hash, self.deck_stack = self.create_deck(num_of_decks)
        self.shuffle_deck(seed)

    #Creates a new ordered stack and a hashmap, modeling a blackjack deck with num_of_decks amount of standard decks
    def create_deck(self, num_of_decks):
        deck_hash = {}
        deck_stack = []
        for card_type in range(1,14):
            deck_hash[card_type] = 4 * num_of_decks
            deck_stack.extend([card_type] * 4 * num_of_decks)
        return deck_hash, deck_stack
    
    #Shuffles the deck_stack using a seed value
    def shuffle_deck(self, seed):
        random_num_generator = np.random.default_rng(seed=seed)
        random_num_generator.shuffle(self.deck_stack)

    #Deals a given number of players and (possibly) a dealer in for blackjack.
    #Returns a table of game data where are players must hit til bust or 21, normal dealer gameplay
    #DeckCounti: i values representing the ratio of each remaining card type in the deck after dealing.
    #DeckSize: the size of the deck after dealing.
    #PlayeriHandProgression and DealerHandProgression: list of all cards dealt to the player at the end of the round.
    #PlayeriOutcome: "Win"/"Loss"/"Draw" for each player's game outcome.
    #PlayeriOptimalStop: The round where the player would've had the most points if stayed
    #DealerFinalScore: Dealer's final score of the round
    #DealerBust: T/F for whether or not the dealer busts that round.
    def play(self, num_of_players=1):
        game_data = None

        while(len(self.deck_stack) > self.penetration * 52):
            player_hands = [[] for _ in range(num_of_players)]

            for player_num in range(0, num_of_players):
                player_hands[player_num].append(self.deck_stack.pop())
                self.deck_hash[player_hands[player_num][-1]] -= 1
                player_hands[player_num].append(self.deck_stack.pop())
                self.deck_hash[player_hands[player_num][-1]] -= 1

            
            dealer_hand = []
            dealer_hand.append(self.deck_stack.pop())
            self.deck_hash[dealer_hand[-1]] -= 1
            dealer_hand.append(self.deck_stack.pop())
            self.deck_hash[dealer_hand[-1]] -= 1


            round_data = self.conductRound(player_hands, dealer_hand)
            if game_data is None:
                game_data = round_data
            else:
                game_data = pd.concat([game_data, round_data], axis=0)
        return game_data

    #conducts one round of blackjack, giving all players a card unless (Dealer blackjack, or player has current score of 21). Updates metrics as needed
    def conductRound(self, player_hands, dealer_hand):
        round_data = {}
        round_data["DeckSize"] = len(self.deck_stack)
        for i in range(1, 14):
            round_data[f"DeckCount{i}"] = self.deck_hash[i] / (self.num_of_decks * 52)
        
        #Precompute dealer's hand
        while (
            21 not in self.getScore(dealer_hand) and
            20 not in self.getScore(dealer_hand) and
            19 not in self.getScore(dealer_hand) and
            18 not in self.getScore(dealer_hand) and
            17 not in self.getScore(dealer_hand) and
            min(self.getScore(dealer_hand)) < 21
        ):
            dealer_hand.append(self.deck_stack.pop())
            self.deck_hash[dealer_hand[-1]] -= 1
        
        dealer_final_score = -1
        for i in range(21, 16, -1):
            if i in self.getScore(dealer_hand):
                dealer_final_score = i
                break
        
        round_data["DealerHandProgression"] = [dealer_hand]
        round_data["DealerFinalScore"] = dealer_final_score
        round_data["DealerBust"] = dealer_final_score == -1

        #Let each player play their round until busting or hitting 21
        for player_hand, player_num in zip(player_hands, range(0, len(player_hands))):
            current_round = 1
            round_data[f"Player{player_num}OptimalStop"] = -1


            #Check if its a blackjack instant win
            if 21 in self.getScore(player_hand) and 21 not in self.getScore(dealer_hand[:2]):
                round_data[f"Player{player_num}Outcome"] = "Win"
                round_data[f"Player{player_num}OptimalStop"] = 0
                round_data[f"Player{player_num}HandProgression"] = [player_hand]
                continue
            
            #Check for a blackjack draw
            elif 21 in self.getScore(player_hand) and 21 in self.getScore(dealer_hand[:2]):
                round_data[f"Player{player_num}Outcome"] = "Draw"
                round_data[f"Player{player_num}OptimalStop"] = 0
                round_data[f"Player{player_num}HandProgression"] = [player_hand]
                continue

            #Check for blackjack instant loss
            elif 21 in self.getScore(dealer_hand[:2]):
                round_data[f"Player{player_num}Outcome"] = "Loss"
                round_data[f"Player{player_num}HandProgression"] = [player_hand]
                continue
            
            #Deal a new cards to the player until they bust or hit 21
            player_max = 0
            while min(self.getScore(player_hand)) < 21 and 21 not in self.getScore(player_hand):
                player_hand.append(self.deck_stack.pop())
                self.deck_hash[player_hand[-1]] -= 1
                winnable_scores = [x for x in self.getScore(player_hand) if x <= 21]
                if len(winnable_scores) > 0 and player_max < max(winnable_scores):
                    round_data[f"Player{player_num}OptimalStop"] = current_round
                    player_max = max(winnable_scores)
                current_round += 1
            
            round_data[f"Player{player_num}HandProgression"] = [player_hand]
            

            
            #Mark round data, and move on to next player
            if 21 not in self.getScore(player_hand):
                round_data[f"Player{player_num}Outcome"] = "Loss"

            elif 21 in self.getScore(player_hand) and dealer_final_score < 21:
                round_data[f"Player{player_num}Outcome"] = "Win"
            
            elif dealer_final_score in self.getScore(player_hand):
                round_data[f"Player{player_num}Outcome"] = "Draw"
        return pd.DataFrame(round_data)
        
    #Gets score from a current hand progression list
    def getScore(self, playerHand):
        scores = []
        score = 0
        for card, i in zip(playerHand, range(0, len(playerHand))):
            if card == 1:
                scores.extend(self.getScore(playerHand[:i] + [11] + playerHand[i+1:]))
            elif card > 10:
                card = 10
            score += card
        scores.append(score)
        return scores

In [0]:
import hashlib

# NORMAL CASINO: 6 decks, 1.5 penetration, 4 players
# bj_game = BlackjackGame(num_of_decks=6, penetration=1.5)
# display(bj_game.play(num_of_players=4))
# game_data.to_csv("single-game-4-player-6-shoe-1.5-penetration.csv", index=True)


#MASSIVE NORMAL CASINO
game_data = None
rng = np.random.default_rng(seed=42)
for i in range(0, 10000):
    print(f"Computing Game {i}")
    seed = rng.integers(0, 100000000)
    bj_game = BlackjackGame(seed=seed)
    if game_data is None:
        game_data = bj_game.play(num_of_players=1)
    else:
        game_data = pd.concat([game_data, bj_game.play(num_of_players=1)], axis=0)
game_data.to_csv("10000-game-4-player-6-shoe-1.5-penetration.csv", index=True)
    
# NORMAL CASINO SOLO: 6 decks, 1.5 penetration, 1 player
# bj_game1 = BlackjackGame(num_of_decks=6, penetration=1.5)
# game_data = bj_game1.play(num_of_players=1)
# game_data.to_csv("single-game-single-player-6-shoe-1.5-penetration.csv", index=True)
